## Data format for MLlib

**Example of Supervised Learning **

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql import Row
# Initialize Spark Session
spark = SparkSession.builder.appName("MLlibDataFormat").getOrCreate()
# Example data
data = [
    Row(label=1.0, features=Vectors.dense([2.5, 3.1, 0.7])),
    Row(label=0.0, features=Vectors.dense([1.2, 0.5, 2.3])),
    Row(label=1.0, features=Vectors.dense([3.4, 2.2, 1.1]))
]
# Create DataFrame
df = spark.createDataFrame(data)
df.show()

+-----+-------------+
|label|     features|
+-----+-------------+
|  1.0|[2.5,3.1,0.7]|
|  0.0|[1.2,0.5,2.3]|
|  1.0|[3.4,2.2,1.1]|
+-----+-------------+



**Feature Transformation for Supervised Learning**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("MLlibFeatureTransformation").getOrCreate()

data = [
    Row(label=500, size=2000, bedrooms=3, age=15),
    Row(label=600, size=2500, bedrooms=4, age=10),
    Row(label=450, size=1800, bedrooms=2, age=20)
]

df = spark.createDataFrame(data)
df.show()

+-----+----+--------+---+
|label|size|bedrooms|age|
+-----+----+--------+---+
|  500|2000|       3| 15|
|  600|2500|       4| 10|
|  450|1800|       2| 20|
+-----+----+--------+---+



**Applying VectorAssembler**

VectorAssembler is a feature transformer in PySpark that combines multiple input columns into a single feature vector column. It is essential for preparing data for machine learning models in PySpark.

In [4]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["size", "bedrooms", "age"], outputCol="features")
df_transformed = assembler.transform(df).select("label", "features")
df_transformed.show(truncate=False)

+-----+-----------------+
|label|features         |
+-----+-----------------+
|500  |[2000.0,3.0,15.0]|
|600  |[2500.0,4.0,10.0]|
|450  |[1800.0,2.0,20.0]|
+-----+-----------------+



In [5]:
spark.stop()

### LIBSVM

simple and efficient text-based format used for storing sparse data in machine learning, especially for Support Vector Machines (SVMs).

Structure of LIBSVM Format - Each line represents one data instance (sample) and follows this format:

label feature_index_1:feature_value_1 feature_index_2:feature_value_2 ... feature_index_n:feature_value_n

-9.490009878824548 1:0.4551273600657362 2:0.36644694351969087 3:-0.38256108933468047 4:-0.4458430198517267 5:0.33109790358914726 6:0.8067445293443565 7:-0.2624341731773887 8:-0.44850386111659524 9:-0.07269284838169332 10:0.5658035575800715


### Linear Regression example with LIBSVM

**Linear Regression** is a statistical method used to model the relationship between a dependent variable (often called the target or response variable) and one or more independent variables

The goal of linear regression is to find the best-fitting straight line (or hyperplane in higher dimensions) that describes the relationship between the independent variables (x) and the dependent variable (y).


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Linear Regression").getOrCreate()
from pyspark.ml.regression import LinearRegression

In [7]:
training = spark.read.format('libsvm').load('Datasets/sample_linear_regression_data.txt')
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

Fit training data to Linear Regression equation


In [8]:
lr = LinearRegression(featuresCol='features', labelCol = 'label', predictionCol='prediction')
lr = LinearRegression()

lrModel = lr.fit(training)

Display coefficient

In [9]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

Display intercept

In [10]:
lrModel.intercept

0.14228558260358093

In [11]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290002,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.695615180432293]


Intercept:0.14228558260358093


In [12]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

In [13]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176535|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
| 2.1228071931912336|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


#### Train/Test split

In [15]:
all_data = spark.read.format('libsvm').load('Datasets/sample_linear_regression_data.txt')
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [17]:
all_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                501|
|   mean|0.25688882219498976|
| stddev| 10.317884030544564|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
+----------

In [18]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
|-15.951512565794573|(10,[0,1,2,3,4,5,...|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-15.310980589416289|(10,[0,1,2,3,4,5,...|
|-14.328978509075442|(10,[0,1,2,3,4,5,...|
|-13.867087895158768|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
|-12.773226999251197|(10,[0,1,2,3,4,5,...|
|-12.558575788856189|(10,[0,1,2,3,4,5,...|
|-12.479280211451497|(10,[0,1,2,3,4,5,...|
|-12.094351278535258|(10,[0,1,2,3,4,5,...|
|-11.904986902675114|(10,[0,1,2,3,4,5,...|
|-11.827072996392571|(10,[0,1,2,3,4,5,...|
|-10.600130341909033|(10,[0,1,2,3,4,5,...|
|-10.489157123372898|(10,[0,1,2,3,4,5,...|
+----------

In [19]:
correct_model = lr.fit(train_data)

In [20]:
test_results = correct_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -21.70933702189673|
| -19.40994823754436|
|-16.142230008176707|
|-17.300780888187393|
|-16.035937773532073|
| -15.90636932514256|
|-18.880603311026857|
|-16.423461329655236|
|-13.886298603280292|
| -16.18536221617946|
|-14.122365127230854|
|-18.042058671527386|
| -8.126040370659826|
|-13.631135813270452|
|-11.256240550348723|
|-10.132964966334775|
| -13.82563923228341|
|-13.113757580005382|
|-13.182719737975788|
|-11.523732486930477|
+-------------------+
only showing top 20 rows



Evaluate the model on the test data


In [21]:
# r2 score
test_results.r2

-0.14515146484322217

In [22]:
# RMSE score
test_results.rootMeanSquaredError

10.234978920441655

#### Deploy: Test the model with unlabeled data

In [23]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [24]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  0.2769492577309219|
|(10,[0,1,2,3,4,5,...|-0.47461253672906417|
|(10,[0,1,2,3,4,5,...| -3.5250886071950136|
|(10,[0,1,2,3,4,5,...|-0.02593984448855452|
|(10,[0,1,2,3,4,5,...|-0.11541157774504096|
|(10,[0,1,2,3,4,5,...|-0.04514324065201365|
|(10,[0,1,2,3,4,5,...|  3.1570876979782883|
|(10,[0,1,2,3,4,5,...|  1.0476036063429377|
|(10,[0,1,2,3,4,5,...| -1.4246819861359976|
|(10,[0,1,2,3,4,5,...|  1.8563837071040181|
|(10,[0,1,2,3,4,5,...| 0.25527723207208614|
|(10,[0,1,2,3,4,5,...|   4.269617109824514|
|(10,[0,1,2,3,4,5,...| -4.6471866285913705|
|(10,[0,1,2,3,4,5,...|  1.0725600244142632|
|(10,[0,1,2,3,4,5,...| -1.2230396611027736|
|(10,[0,1,2,3,4,5,...| -1.9613863122004842|
|(10,[0,1,2,3,4,5,...|  1.9206523296082956|
|(10,[0,1,2,3,4,5,...|   1.286684583612811|
|(10,[0,1,2,3,4,5,...|  2.5825893960667554|
|(10,[0,1,2,3,4,5,...|  1.034575

##### Stop Spark

In [25]:
spark.stop()

### Analysis of E-Commerce Customer

Data Description

Email - Unique identifier for the customer (not used in regression).

Address - Customer's physical address (not used in regression).

Avatar - Link or identifier for the customer's profile picture (not used in regression).

Avg Session Length - The average duration of a customer session (measured in minutes).

Time on App - Total time spent on the mobile app (measured in minutes).

Time on Website - Total time spent on the website (measured in minutes).

Length of Membership - How long a customer has been a member (measured in years).

Yearly Amount Spent - Target variable: The total amount the customer spends per year (measured in dollars).

In [26]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Ecommerce Linear Regression").getOrCreate()

In [27]:
from pyspark.ml.regression import LinearRegression

In [28]:
data = spark.read.csv('Datasets/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [29]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [30]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [31]:
# Transform the data for MLlib (VectorAssembler)

# from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [32]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [33]:
assembler = VectorAssembler(inputCols=
 ['Avg Session Length',
  'Time on App',
  'Time on Website',
  'Length of Membership'], outputCol='features')

Transform columns into features

In [34]:
output = assembler.transform(data)
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [35]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [36]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



Features and Label

In [37]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

Make the final data for MLlib

In [38]:
final_data = output.select('features','Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

Train Test split

In [39]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [40]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean| 501.11057130393704|
| stddev|  81.98986477383264|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [41]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean|  494.9156297682605|
| stddev|   72.4313795369055|
|    min|  304.1355915788555|
|    max|  725.5848140556806|
+-------+-------------------+



Linear Regression

In [42]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')
lr_model = lr.fit(train_data)

In [43]:
lr_model.coefficients

DenseVector([25.9338, 38.9702, 0.0422, 61.3812])

In [44]:
lr_model.intercept

-1045.931344483102

Evaluate the model

In [45]:
test_results = lr_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.567066025407485|
|-11.753917369101032|
|-12.664493117061681|
|-21.724664543914628|
| -7.100004026618308|
| 19.856445408852835|
| 3.9038272067908224|
| -17.93364662331419|
| 16.540915100689347|
|  6.725162962410138|
|  2.623069345821534|
| -2.372354738268257|
|  8.041724118929949|
| -9.271203337255827|
|  7.817071681578284|
|-10.927445115446858|
| -2.257198279421061|
|   8.74665775072856|
| -17.78042657530682|
|  5.152160002416849|
+-------------------+
only showing top 20 rows



In [46]:
test_results.rootMeanSquaredError

10.292559813303104

In [47]:
test_results.r2

0.9796671247860095

Statistic about the data

In [48]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



Deployment

In [49]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.3123495994443...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.7366356860502...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.9120759292006...|
|[31.9549038566348...|
|[31.9563005605233...|
|[31.9764800614612...|
+--------------------+
only showing top 20 rows



In [50]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|   397.07328504722|
|[30.3931845423455...|331.68278717229464|
|[31.0662181616375...|461.59778632473603|
|[31.1239743499119...| 508.6717183836804|
|[31.1280900496166...|  564.352690773673|
|[31.3123495994443...| 443.7349726190878|
|[31.5316044825729...|432.61177852257174|
|[31.5702008293202...| 563.8791387647191|
|[31.6098395733896...| 428.0046345504188|
|[31.6548096756927...| 468.5382607651384|
|[31.7366356860502...| 494.3103769097104|
|[31.8186165667690...| 448.7910281084039|
|[31.8209982016720...| 416.6335568942834|
|[31.8279790554652...|449.27395088419735|
|[31.8512531286083...| 465.1751749852201|
|[31.8648325480987...|450.81872559226053|
|[31.9120759292006...| 389.7919145851288|
|[31.9549038566348...| 431.2512221891984|
|[31.9563005605233...| 564.9063583225056|
|[31.9764800614612...|325.44228603168335|
+--------------------+------------

Make predictions on the new unseen data

In [51]:
from pyspark.ml.linalg import DenseVector
from pyspark.sql import Row
# Example of unseen data point
new_data = [Row(features=DenseVector([31.0472, 11.1997, 38.6887, 3.0888]))]

# Convert it into a PySpark DataFrame
unseen_df = spark.createDataFrame(new_data)

# Show the new test data
unseen_df.show(truncate=False)

+--------------------------------+
|features                        |
+--------------------------------+
|[31.0472,11.1997,38.6887,3.0888]|
+--------------------------------+



In [52]:
predictions = lr_model.transform(unseen_df)

# Show results
predictions.select("features", "prediction").show(truncate=False)

+--------------------------------+-----------------+
|features                        |prediction       |
+--------------------------------+-----------------+
|[31.0472,11.1997,38.6887,3.0888]|386.9210666237934|
+--------------------------------+-----------------+

